In [1]:
import pandas as pd #used for constructing dataframes
import numpy as np
from multiprocessing import Pool #Multithreading pysam read lookup 
import multiprocessing as mp #Multithreading for metaplot summation
import pickle #used to save generated plots
import re
import os

#plotting with Bokeh:
from bokeh.io import output_notebook, export_png, export_svg
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, NumeralTickFormatter, ContinuousTicker
from bokeh.layouts import gridplot
from bokeh.transform import linear_cmap
import colorcet as cc
output_notebook()

#adding iPython interaction to bokeh plots
from IPython.display import display

#Reading bam files
import pysam

from time import strftime

prefix = "minopt"
graphs = []

Loading BokehJS ...

## Loading GFF annotations and filtering for Yeast
This cell loads a GFF file containing annotations and filters them for a subset of transcripts to use in metagene averaging. 

In principle any GFF file can be used with some modification to this cell. The pandas column labels that are necessary downstream are: 
    "feature": containing the values "transcript", "CDS", and "exon"
    "transcript_id": containing an ID that associates "CDS" and "exon" features to a particular "transcript" feature
    "start": containing the first position of the feature (inclusive)
    "end": containing the last position of the feature (inclusive)
    "strand": containing the value "+" or "-"


In [2]:
pickled_gtf_df = False  #set to False to load from GTF file, set to filepath to restore pickled gtf_df
if not pickled_gtf_df:
    print("No pickled gtf_df set. Loading from GTF file")
    with open("/home/anthony/minreporter_profiling/annotations_from_colin/reporter_genome_annos_with_UTRS/OPT.gtf", "r") as inGTF:
        gtf_dict = {"chrom":[], "feature":[], "start":[], "end":[], 
                    "gene_id":[], "transcript_id":[], "strand":[],
                   }
        for line in inGTF:
            try:
                if not line.startswith("#"):
                    splitline = line.split("\t")
                    chrom = splitline[0]
                    if chrom == "chrM":
                        chrom = "chrMito"
                    feature = splitline[2]
                    if feature == "stop_codon": 
                        feature = "CDS"
                    if feature not in ["transcript", "CDS", "exon"]:
                        continue
                    start = int(splitline[3])
                    end = int(splitline[4])
                    gene_id = re.search('gene_id "(.*?)";', splitline[8]).group(1)
                    transcript_id = re.search('transcript_id "(.*?)";', splitline[8]).group(1)
                    annotation_type = splitline[1]
                    strand = splitline[6]
                    gtf_dict["chrom"].append(chrom)
                    gtf_dict["feature"].append(feature)
                    gtf_dict["start"].append(start)
                    gtf_dict["end"].append(end)
                    gtf_dict["gene_id"].append(gene_id)
                    gtf_dict["transcript_id"].append(transcript_id)
                    gtf_dict["strand"].append(strand)
            except:
                print(line)
                raise
    gtf_df = pd.DataFrame(gtf_dict)
    transcript_lines = gtf_df[gtf_df.feature == "exon"].replace("exon", "transcript")
    def define_transcript_for_spliced(group):
        new_start = group.start.min()
        new_end = group.end.max()
        group.drop_duplicates(subset="transcript_id", inplace=True)
        group.start = new_start
        group.end = new_end
        return group
    transcript_lines = transcript_lines.groupby("transcript_id").apply(define_transcript_for_spliced)
    gtf_df = pd.concat([transcript_lines, gtf_df], axis=0)
    

    print("GTF loaded:", len(gtf_df[gtf_df.feature == "transcript"]), "transcripts")


    filename = "./parsed_gtf_"+strftime("%Y%m%d%H%M%S")+"_deduplicated.pkl"
    pickle.dump(gtf_df, open(filename, "wb"))
    print("Pickling gtf_df to",filename)

else:
    #load pickled gtf_df
    gtf_df = pickle.load(open(pickled_gtf_df, "rb"))
    print("Pickled gtf_df loaded:", len(gtf_df[gtf_df.feature == "transcript"]), "transcripts")
    #load dict of dicts containing spliced CDS coordinates for each gene
    # e.g. {"ENST00000321256":{"ATG":[1, 67, 160, ...], "ACG":[16, 19, ...], ...}, ...}
    #codon_positions = pickle.load(open("/home/anthony/test_code/global_codon_positions.pkl", "rb"))
    all_codons_df = pickle.load(open("../all_yeast_codons_table.pkl", "rb"))
    print("Loaded codon positions pkl")

Pickled gtf_df loaded: 6693 transcripts
Loaded codon positions pkl


## Multithreaded density addition

### Define density calculation and summation functions
```count_density_2``` is the workhorse of the script that calculates normalized densities for each transcript  
```sum_density sums``` up all the results of ```count_density_2```

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'
def get_density(transcript, file_path):
    '''takes in a transcript line of gtf_df and creates a tx_density object. These will then be saved to a pkl file for fast parsing of downstream calculations.
    '''
    global prefix
    global gtf_df
    bamfile = pysam.AlignmentFile(file_path)
    
    chrom = transcript[1].chrom
    strand = transcript[1].strand
    transcript_id = transcript[1].transcript_id
    tx_cds_exons_df = gtf_df[gtf_df.transcript_id == transcript_id] #forms a dataframe containing transcript, CDS, 
                                                                                  # and exon entries from gtf_df
    
    tx = tx_cds_exons_df[tx_cds_exons_df.feature == "transcript"] #transcript line(s) of dataframe
    tx_left = tx.start.min() #Leftmost genome coordinate of transcript (could be 3' or 5' end depending on strand)
    tx_right = tx.end.max()  #Rightmost genome coordinate of transcript

    exons = tx_cds_exons_df[tx_cds_exons_df.feature == "exon"] #Exon lines of dataframe
    tx_len = sum([(exon[1].end + 1) - exon[1].start for exon in exons.iterrows()]) #length of spliced transcript feature
    
    cds = tx_cds_exons_df[tx_cds_exons_df.feature == "CDS"] #CDS lines of dataframe
    cds_left_genomecoords = cds.iloc[0].start
    cds_right_genomecoords = cds.iloc[-1].end
    
    #creates a dataframe of all information on reads mapping anywhere in the transcript.
    read_iter = bamfile.fetch(chrom, int(tx_left), int(tx_right), multiple_iterators=True)
    read_dict = {"read_id":[], "length":[], "left_pos":[], "right_pos":[], "transcript_id":[]}
    for read in read_iter:
        if (read.is_reverse and strand == "-") or (not read.is_reverse and strand == "+"):
            read_dict["read_id"].append(read.query_name)
        else:
            continue
        if strand == "+" and read.cigartuples[0] == (4,1): #(4,1) corresponds to a cigar code of 1S
            read_dict["left_pos"].append(read.reference_start + 1 + 1) #pysam fetches coordinates 0-based, +1 to make 1-based, +1 to correct for untemplated RT addition
            read_dict["right_pos"].append(read.reference_end) #this gives nt 1 past end, so -1 to get end and +1 to make 1-based
            read_dict["length"].append(read.query_length - 1) #-1 to correct for untemplated RT addition
        elif strand == "-" and read.cigartuples[-1] == (4,1):
            read_dict["left_pos"].append(read.reference_start + 1) #pysam fetches coordinates 0-based, +1 to make 1-based
            read_dict["right_pos"].append(read.reference_end - 1) #this gives nt 1 past end, so -1 to get end and +1 to make 1-based, -1 to correct for untemplated RT addition
            read_dict["length"].append(read.query_length - 1)
        else:
            read_dict["left_pos"].append(read.reference_start + 1) #pysam fetches coordinates 0-based, +1 to make 1-based
            read_dict["right_pos"].append(read.reference_end) #this gives nt 1 past end, so -1 to get end and +1 to make 1-based
            read_dict["length"].append(read.query_length) #-1 to correct for untemplated RT addition
        read_dict["transcript_id"].append(transcript_id)
    read_df = pd.DataFrame.from_dict(read_dict) 
    #read_df = all_read_df
    
    #creates two pd.Series to translate genome coordinates into spliced/unspliced transcript coords.
    unspliced_tx_len = tx_right - tx_left + 1
    exonic_positions = [] #pd.Series to translate genome coords to spliced transcript coords
    
    for exon in exons.iterrows():
        exonic_positions.extend(np.arange(exon[1].start, exon[1].end+1))
    exonic_positions = pd.Series(np.arange(1,len(exonic_positions)+1), index=exonic_positions) 
    cds_left = exonic_positions[cds_left_genomecoords]
    cds_right = exonic_positions[cds_right_genomecoords]
    global x
    x = (cds_left, cds_right, tx_len)
    if strand == "+":
        start_aligned = np.arange(-(cds_left-1), tx_len-cds_left+1)
        stop_aligned = np.arange(-(cds_right-1), tx_len-cds_right+1)
        transcript_positions = pd.Series(np.arange(1, unspliced_tx_len+1), index=np.arange(tx_left, tx_right+1))
    elif strand == "-":
        start_aligned = np.arange(cds_right-tx_len, cds_right)[::-1]
        stop_aligned = np.arange(cds_left-tx_len, cds_left)[::-1]
        #start_aligned = np.arange(-(cds_right-1), tx_len-cds_right+1)[::-1]
        #stop_aligned = np.arange(-(cds_left-1), tx_len-cds_left+1)[::-1]
        transcript_positions = pd.Series(np.arange(1, unspliced_tx_len+1)[::-1], index=np.arange(tx_left, tx_right+1))
    try:
        exonic_positions_start = pd.Series(start_aligned, index=exonic_positions.index)
        exonic_positions_stop = pd.Series(stop_aligned, index=exonic_positions.index)
    except:
        print(transcript_id)
        raise
    
    
    #only select reads contained entirely within annotated transcript and not contained entirely in exons
    unspliced_reads = read_df.loc[
        ((read_df.left_pos.isin(transcript_positions.index)) & 
         (read_df.right_pos.isin(transcript_positions.index)))] 
    tx_aligned_reads = read_df.loc[
        ((read_df.left_pos.isin(transcript_positions.index)) & 
         (read_df.right_pos.isin(transcript_positions.index)))].loc[:,["read_id", "length", "transcript_id"]]
    #all reads transformed to unspliced transcript coords
    if strand == "+":
        unspliced_reads["5p_pos_tx"] = transcript_positions[unspliced_reads.left_pos].values
        unspliced_reads["3p_pos_tx"] = transcript_positions[unspliced_reads.right_pos].values
    elif strand == "-":
        unspliced_reads["5p_pos_tx"] = transcript_positions[unspliced_reads.right_pos].values
        unspliced_reads["3p_pos_tx"] = transcript_positions[unspliced_reads.left_pos].values
    
    
    #only select reads that are contained entirely within introns
    exonic_reads = read_df.loc[
        (read_df.left_pos.isin(exonic_positions.index)) & 
        (read_df.right_pos.isin(exonic_positions.index))]
    cds_aligned_reads = read_df.loc[
        (read_df.left_pos.isin(exonic_positions.index)) & 
        (read_df.right_pos.isin(exonic_positions.index))].loc[:,["read_id"]]
    
    
    #change from genome coords to start aligned transcript coords
    if strand == "+":
        cds_aligned_reads["5p_pos_start_cds"] = exonic_positions_start[exonic_reads.left_pos].values
        cds_aligned_reads["3p_pos_start_cds"] = exonic_positions_start[exonic_reads.right_pos].values
        cds_aligned_reads["5p_pos_stop_cds"] = exonic_positions_stop[exonic_reads.left_pos].values
        cds_aligned_reads["3p_pos_stop_cds"] = exonic_positions_stop[exonic_reads.right_pos].values
    elif strand == "-":
        cds_aligned_reads["5p_pos_start_cds"] = exonic_positions_start[exonic_reads.right_pos].values
        cds_aligned_reads["3p_pos_start_cds"] = exonic_positions_start[exonic_reads.left_pos].values
        cds_aligned_reads["5p_pos_stop_cds"] = exonic_positions_stop[exonic_reads.right_pos].values
        cds_aligned_reads["3p_pos_stop_cds"] = exonic_positions_stop[exonic_reads.left_pos].values
    cds_aligned_reads["exonic"] = [True] * len(exonic_reads)
    
    #create dataframe containing all reads for transcript.
    read_df = pd.merge(cds_aligned_reads, unspliced_reads, on=["read_id"], suffixes=["_cds", "_tx"], how="outer")
    #read_df.transcript_id_tx.fillna(read_df.transcript_id_unspliced, inplace=True)

    
    #makes a list (called fivep or threep) containing all the read ends for transcript.
    # This has to take strandedness into account to determine 3' or 5' end.
    # numpy.histogram is used to turn list of ends into mapped-end depth at each position.
    tx_obj = pd.DataFrame()
    tx_obj["transcript_id"] = [transcript_id]
    tx_obj["cds_left_spliced"] = [cds_left]
    tx_obj["cds_right_spliced"] = [cds_right]
    tx_obj["cds_left_unspliced"] = [cds_left_genomecoords - tx_left + 1]
    tx_obj["cds_right_unspliced"] = [cds_right_genomecoords - tx_left + 1]
    tx_obj["strand"] = strand
    tx_obj["spliced_len"] = tx_len
    tx_obj["chrom"] = chrom
    
    
    #create and return a tx_density object that contains all the read ends
    
    return((read_df, tx_obj))

In [4]:
#%timeit -n1 -r1



def run_and_pickle(bam_file_path, pickle_path):
    global all_read_df
    global all_properties_df
    global codon_positions
        
    downsample_frac = 1

    p = Pool(96)
    transcript_rows = gtf_df[gtf_df.feature == "transcript"].sample(frac=downsample_frac, axis=0)
    args = zip(transcript_rows.iterrows(), [bam_file_path]*len(transcript_rows))
    map_out = p.starmap(get_density, args)
    p.close()

    all_read_df = pd.concat([entry[0] for entry in map_out], axis=0, sort=False).reset_index(drop=True)
    all_properties_df = pd.concat([entry[1] for entry in map_out], axis=0, sort=False).reset_index(drop=True)
    group = all_read_df.groupby("transcript_id")
    read_count = group.exonic.transform("count")
    all_read_df["len_read_percent"] = 1/read_count

    out_dict = {}
    pickle.dump({"tx_reads":all_read_df, "tx_properties":all_properties_df}, 
            open(pickle_path, "wb"))
    print("Pickled", prefix, "sample to "+pickle_path)

def load_data_from_pickle(pickle_path):
    global all_read_df
    global all_properties_df
    in_df = pickle.load(open(pickle_path, "rb"))
    all_read_df = in_df["tx_reads"]
    all_properties_df = in_df["tx_properties"]
    print("loaded pickle", pickle_path)

sample_names = ["wt_100_mono_r2", "wt_100_di_r1", "wt_100_di_r2"]#, "wt_100_mono_r1", ]

for sample_name in sample_names:
    pickle_path = f"./{sample_name}_deduplicated.pkl" #"./genome_aligned_reads.pkl"
    bam_file_path = f"/home/anthony/minreporter_profiling/output/round2/pipeline_output_minopt/output/genome_aligned/{sample_name}/{sample_name}_Aligned.out.sorted.bam"


In [5]:
'''x = all_read_df.groupby(by="transcript_id_unspliced", as_index=False)
p = Pool(96)
new_list = p.starmap(myapply, zip(x[1], itertools.repeat(50), itertools.repeat(50), itertools.repeat("start")))
p.close
new_df = pd.concat(new_list, axis=0)'''

def calculate_shifts(mapped_end, start_or_stop, align_to_site, pickle_in):
    in_df = pickle.load(open(pickle_in, "rb"))
    all_read_df = in_df["tx_reads"]
    all_properties_df = in_df["tx_properties"]
    print("loaded pickle", pickle_in, "for shift calculation.")

    lengths = all_read_df.length.drop_duplicates()
    length_mode = float(all_read_df.length.mode())

    column_name = mapped_end+"_pos_"+start_or_stop+"_cds"
    plot_range = np.arange(-nt_upstream, nt_downstream + 1)


    gene_lengths = [1]*(nt_upstream+nt_downstream)

    #histogramming
    plotdf = all_read_df[all_read_df.length.isin(lengths)].dropna()
    group_histo = all_read_df.dropna().groupby("length").apply(lambda x: np.histogram(x[column_name], plot_range)[0]/gene_lengths) #weights=x.len_read_percent
    group_histo = pd.DataFrame(group_histo.to_dict())
    group_histo.index = plot_range[:-1]
    #histo = np.histogram(plotdf["5p_pos_start_cds"], np.arange(-50, 501), weights=plotdf.len_read_percent)[0]

    #riboshifting
    group_histo_subset = group_histo.loc[np.arange(-20,20), [length_mode]]
    group_histo_mode = group_histo_subset.loc[:,[length_mode]].idxmax().values[0]
    mode = group_histo_mode #group_histo.sum(axis=1).idxmax()
    print("Highest peak is", mode, "from annotated start codon")
    shift_dict = { #+1s to account for digestion heterogeneity
        "p_site":-mode + 1,
        "a_site":-mode + 3 + 1,
        "e_site":-mode - 3 + 1
    }
    return (shift_dict, group_histo)

#plots
def plot_readlen_distribs(pickle_in, shift_dict, group_histo, show_plots=True):
    in_df = pickle.load(open(pickle_in, "rb"))
    all_read_df = in_df["tx_reads"]
    all_properties_df = in_df["tx_properties"]
    mapper = linear_cmap(field_name='values', palette=cc.b_linear_blue_95_50_c20, low=0, high=group_histo.max().max())
    y = [yval for xval in group_histo.index for yval in group_histo.columns]
    x = [xval for xval in group_histo.index + shift_dict[align_to_site] for yval in group_histo.columns]
    values = [group_histo.loc[xval,yval] for xval in group_histo.index for yval in group_histo.columns]
    plot_df = pd.DataFrame({"x":x, "y":y, "values":values})
    p = figure(width=1400, title=pickle_in)
    p.add_tools(HoverTool(tooltips=[
        ("Read length", "@y"),
        ("Position (vs. start)", "@x"),
        ("Height", "@values")
    ]))
    p.rect(x="x", y="y", color=mapper, line_color=mapper, width=1, height=1, source=plot_df, line_width=0.5)

    p2 = figure(title="Size distribution of mapped reads for "+pickle_in, x_axis_label="Read length", y_axis_label="Count")
    plot_df = pd.DataFrame(all_read_df[all_read_df.exonic == True].groupby("length").read_id.count().rename("number"))
    plot_df.number = plot_df.number#/np.sum(plot_df.number)
    p2.line(x="length", y="number", line_width=2, source=plot_df)
    p2.add_tools(HoverTool(tooltips=[
        ("Read length", "@length"),
        ("Count", "@number"),
    ]))
    p2.xaxis.major_label_text_font_size = "12pt"
    p2.yaxis.major_label_text_font_size = "12pt"
    p2.xaxis.axis_label_text_font_size = "15pt"
    p2.yaxis.axis_label_text_font_size = "15pt"

    if show_plots:
        grid = gridplot([[p],[p2]])
        show(grid)
    else:
        return (p, p2)
    
    
nt_upstream = 50
nt_downstream = 70
mapped_end = "5p" # to get true ribosome shifts, use 5p aligned.
start_or_stop = "start"
align_to_site = "a_site"
    
sample = "wt_non_mono_r3" # change sample name to run analysis for that sample
pickle_in = f"./{sample}.pkl"

shift_dict, group_histo = calculate_shifts(mapped_end, start_or_stop, align_to_site, pickle_in)
plot_readlen_distribs(pickle_in, shift_dict, group_histo, show_plots=True)

loaded pickle ./wt_non_mono_r3.pkl for shift calculation.
Highest peak is -13 from annotated start codon


In [6]:
shift_dict = {'p_site': 14, 'a_site': 17, 'e_site': 11}

{'p_site': 14, 'a_site': 17, 'e_site': 11}

## 21/28 on reporter

In [6]:
rep_reads = all_read_df[(all_read_df.exonic == True) & ~(all_read_df.transcript_id == transcript)]

NameError: name 'all_read_df' is not defined

In [20]:
#igv plotting
transcript="his3_rep"


load_data_from_pickle("../round2/wt_100_mono_r2_deduplicated.pkl")
p2 = figure(title="Size distribution of mapped reads for "+"wt_100_r1", x_axis_label="Read length", y_axis_label="Count", x_range=(14,40))
plot_df = pd.DataFrame(all_read_df[(all_read_df.exonic == True) & (all_read_df.transcript_id == transcript) & (all_read_df.length <= 36)])
plot_df = plot_df[(plot_df["5p_pos_start_cds"]+17 > 27) & ~(plot_df["5p_pos_start_cds"]+17).isin(range(361,384))] # exclude flag tag and primer region
plot_df = plot_df[(plot_df["5p_pos_stop_cds"]+17 < -15)] # remove 5 codons around stop codon
len_plot_df = plot_df.groupby("length").read_id.count().reset_index()
len_plot_df.columns = ["length", "number"]
len_plot_df.number = len_plot_df.number/np.sum(len_plot_df.number)

non_all_read_df = pickle.load(open("../round3/wt_non_mono_r3.pkl", "rb"))["tx_reads"]
non_plot_df =  pd.DataFrame(non_all_read_df[(non_all_read_df.exonic == True) & (non_all_read_df.transcript_id == transcript) & (non_all_read_df.length <= 36)])
non_plot_df = non_plot_df[(non_plot_df["5p_pos_start_cds"]+17 > 27) & ~(non_plot_df["5p_pos_start_cds"]+17).isin(range(361,384))] # exclude flag tag and primer region
non_plot_df = non_plot_df[(non_plot_df["5p_pos_stop_cds"]+17 < -15)] # remove 5 codons around stop codon
len_non_plot_df = non_plot_df.groupby("length").read_id.count().reset_index()
len_non_plot_df.columns = ["length", "number"]
len_non_plot_df.number = len_non_plot_df.number/np.sum(len_non_plot_df.number)

p2.line(x="length", y="number", line_width=2, source=len_plot_df, legend_label="opt")
p2.line(x="length", y="number", line_width=2, source=len_non_plot_df, color="red", legend_label="non")

p2.add_tools(HoverTool(tooltips=[
    ("Read length", "@length"),
    ("Count", "@number"),
]))



p2.xaxis.major_label_text_font_size = "12pt"
p2.yaxis.major_label_text_font_size = "12pt"
p2.xaxis.axis_label_text_font_size = "15pt"
p2.yaxis.axis_label_text_font_size = "15pt"
show(p2)

loaded pickle ../round2/wt_100_mono_r2_deduplicated.pkl


# Calculate normalized 21/28 ratios

In [16]:
load_data_from_pickle("../round2/wt_100_mono_r2_deduplicated.pkl")
non_all_read_df = pickle.load(open("./wt_non_mono_r3.pkl", "rb"))["tx_reads"]

loaded pickle ../round2/wt_100_mono_r2_deduplicated.pkl


In [11]:
opt_riboshift = 17
non_riboshift = 17

opt_total_plot_df = all_read_df.groupby("length").read_id.count().reset_index()
opt_total_plot_df.columns = ["length", "number"]
plot_df = pd.DataFrame(all_read_df[(all_read_df.exonic == True) & (all_read_df.transcript_id == transcript)])
plot_df = plot_df[(plot_df["5p_pos_start_cds"]+opt_riboshift > 27) & ~(plot_df["5p_pos_start_cds"]+opt_riboshift).isin(range(361,384))] # exclude flag tag and primer region
plot_df = plot_df[(plot_df["5p_pos_stop_cds"]+opt_riboshift < -15)] # remove 5 codons around stop codon
len_plot_df = plot_df.groupby("length").read_id.count().reset_index()
len_plot_df.columns = ["length", "number"]

non_total_plot_df = non_all_read_df.groupby("length").read_id.count().reset_index()
non_total_plot_df.columns = ["length", "number"]
non_plot_df =  pd.DataFrame(non_all_read_df[(non_all_read_df.exonic == True) & (non_all_read_df.transcript_id == transcript)])
non_plot_df = non_plot_df[(non_plot_df["5p_pos_start_cds"]+non_riboshift > 27) & ~(non_plot_df["5p_pos_start_cds"]+non_riboshift).isin(range(361,384))] # exclude flag tag and primer region
non_plot_df = non_plot_df[(non_plot_df["5p_pos_stop_cds"]+non_riboshift < -15)] # remove 5 codons around stop codon
len_non_plot_df = non_plot_df.groupby("length").read_id.count().reset_index()
len_non_plot_df.columns = ["length", "number"]


#for rep2 ranges
opt_total_21mer = opt_total_plot_df.query("20 <= length <= 25").number.sum()
opt_total_28mer = opt_total_plot_df.query("27 <= length <= 36").number.sum()
opt_21mer = len_plot_df.query("20 <= length <= 25").number.sum()
opt_28mer = len_plot_df.query("27 <= length <= 36").number.sum()

non_total_21mer = non_total_plot_df.query("20 <= length <= 26").number.sum()
non_total_28mer = non_total_plot_df.query("27 <= length <= 39").number.sum()
non_21mer = len_non_plot_df.query("20 <= length <= 26").number.sum()
non_28mer = len_non_plot_df.query("(27 <= length <= 39)").number.sum()

print("opt 21/28 ratio: ", round(opt_21mer/opt_28mer/(opt_total_21mer/opt_total_28mer), 3))
print("non 21/28 ratio: ", round(non_21mer/non_28mer/(non_total_21mer/non_total_28mer), 3))

opt 21/28 ratio:  1.148
non 21/28 ratio:  1.464


### Barplot

In [21]:
ratios_21_28 = {
    "opt_r1": 1.148,
    "non_r1": 1.551,
    "opt_r2": 1.026,
    "non_r2": 1.443,
    "non_r3": 1.464,
}

ratios_21_28 = pd.DataFrame(ratios_21_28, index=[1])

In [30]:
vbar_data = ratios_21_28
#vbar_data.loc[:,['opt_r1', 'non_r1', 'opt_r2', 'non_r2']] = 1/vbar_data[['opt_r1', 'non_r1', 'opt_r2', 'non_r2']]
opt_mean = vbar_data.loc[1,["opt_r1", "opt_r2"]].mean()
non_mean = vbar_data.loc[1,["non_r1", "non_r2", "non_r3"]].mean()
opt_std = vbar_data.loc[1,["opt_r1", "opt_r2"]].std()
non_std = vbar_data.loc[1,["non_r1", "non_r2", "non_r3"]].std()


p = figure(x_range=("minOPT", "minNONOPT"), y_range=(0,1.75), plot_height=400, plot_width=350, output_backend="svg")
p.vbar(x=["minOPT", "minNONOPT"], top=[opt_mean, non_mean],
      color="#cfcfcf", width=.75)
p.circle(x=["minOPT"]*2, y=vbar_data.loc[1,["opt_r1", "opt_r2"]], color="gray", size=7)
p.circle(x=["minNONOPT"]*3, y=vbar_data.loc[1,["non_r1", "non_r2", "non_r3"]], color="gray", size=7)
p.dash(x=["minOPT"]*2, y=[opt_mean+opt_std, opt_mean-opt_std], size=15, color="gray")
p.dash(x=["minNONOPT"]*2, y=[non_mean+non_std, non_mean-non_std], size=15, color="gray")
p.line(x=["minOPT"]*2, y=[opt_mean+opt_std, opt_mean-opt_std], color="gray")
p.line(x=["minNONOPT"]*2, y=[non_mean+non_std, non_mean-non_std], color="gray")

p.title.text = "Normalized 21mer/28mer ratio"
p.yaxis.axis_label = "Normalized 21mer/28mer ratio"

p.grid.grid_line_alpha = 0
p.yaxis.axis_label_text_font_size = "13pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "13pt"
p.yaxis.minor_tick_line_alpha = 0

show(p)
export_svg(p, filename="./paper_plots/ratio_21_28_on_reporter.svg")

['./paper_plots/ratio_21_28_on_reporter.svg']

In [287]:
## IGV View for comparing 21mer and 28mer, etc.

for sample in ["wt_100_mono_r1", "wt_100_mono_r2", "wt_0_mono_r1"]:

    transcript = "his3_rep"
    shift = 13
    load_data_from_pickle(f"./{sample}.pkl")

    plot_df = pd.DataFrame(all_read_df[(all_read_df.exonic == True) & (all_read_df.transcript_id == transcript)])
    reads_16mer = plot_df.query("14 <= length <= 17")
    reads_21mer = plot_df.query("20 <= length <= 25")
    reads_28mer = plot_df.query("27 <= length <= 36")

    tx_df = gtf_df[gtf_df.transcript_id == transcript]
    tx = tx_df[tx_df.feature == "transcript"]
    exons = tx_df[tx_df.feature == "exon"]
    CDS = tx_df[tx_df.feature == "CDS"]
    strand = tx.strand.values
    if strand == "+":
        start_codon = CDS.iloc[0].start
        stop_codon = CDS.iloc[-1].end
    elif strand == "-":
        start_codon = CDS.iloc[-1].end
        stop_codon = CDS.iloc[0].start


    hover = HoverTool(tooltips=[
        ("Reads", "@top"),
        ("Position", "@right")
    ], mode="vline", attachment="above", anchor="top_center")

    hover2 = HoverTool(tooltips=[
        ("Position", "@x")
    ], name="triangle")


    #shift using riboshift from previous cell
    for df in [reads_16mer, reads_21mer, reads_28mer]:
        df.loc[:,"left_pos"] = df["left_pos"] + shift

    histo_df_list = []
    for idx, tx_reads in enumerate([reads_28mer, reads_21mer, reads_16mer]):
        if len(tx_reads) > 0:
            histo = np.histogram(tx_reads.left_pos, bins=np.arange(tx.start.item(), tx.end.item()+1), weights=tx_reads.len_read_percent)
            histo_df = pd.DataFrame({"top":histo[0], "left":histo[1][:-1], "right":histo[1][1:], "bottom":[0]*len(histo[0])})
            histo_df = histo_df[histo_df.top != 0]
            range_upper = histo_df.top.max()
        else:
            histo_df = pd.DataFrame()
            range_upper = 1
        histo_df_list.append(histo_df)

    y_upper = 0
    for histo_df in histo_df_list:
        y_upper = max(y_upper, histo_df.top.max())

    read_p = figure(height=300, width=1500, y_range=(0,y_upper), output_backend="webgl", tools=[hover, "xbox_zoom"], title=sample)#, y_axis_type="log", y_range = (0.1, range_upper))
    for idx, histo_df in enumerate(histo_df_list):

        read_p.quad(left="left", right="right", top="top", bottom="bottom", color=["blue", "orange", "red"][idx], source=histo_df, legend_label=["28mer", "21mer", "16mer"][idx])
        read_p.xaxis.ticker = []
        read_p.xgrid.grid_line_color = None
        read_p.title.text_font_size = "20pt"
        read_p.y_range
        read_p.title.text_font_size = "18pt"

    p = figure(height=100, width=1500, y_range=(-3,3), tools=["xbox_zoom", "reset", hover2], output_backend="webgl")
    p.x_range = read_p.x_range
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.yaxis.ticker = []
    p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    p.line(x=list(map(int, [tx.start, tx.end+1])), y=[0, 0], color="purple", line_width=5, level="glyph")
    p.quad(left=exons.start, right=exons.end+1, top=1, bottom=-1, color="white", line_color="purple", line_width=2, level="annotation")
    p.quad(left=CDS.start, right=CDS.end+1, top=1, bottom=-1, color="purple", line_color=None, level="annotation")
    p.inverted_triangle(x="x", y="y", color="green", size=10, source=pd.DataFrame({"x":[start_codon], "y":[2]}))
    p.inverted_triangle(x="x", y="y",  color="red", size=10, source=pd.DataFrame({"x":[stop_codon+1], "y":[2]}))


    p.quad(left=stop_codon, right=stop_codon+1, top=1, bottom=-1, color="red", level="annotation")


    grid = gridplot([[read_p],[p]])
    show(grid)
    #export_png(grid, filename=f"./igv_plots/{sample}_igv_plot.png")
    print("exported", sample)

/home/anthony/software/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "bottom" value "bottom", key "left" value "left", key "right" value "right", key "top" value "top" [renderer: GlyphRenderer(id='81959', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "bottom" value "bottom"

exported wt_100_mono_r1


# Plots for paper

## Monosomes

In [72]:
total_mapped_reads = {
    "wt_100_mono_r1":15340700,
    "wt_0_mono_r1": 7134795,
    "wt_100_mono_r2": 8604380
}

## IGV View
# this has the problem for minus strand reads that it will use the "left side" of the read instead of the 5' end. And it shows reads that 
#   aligned on both strands, rather than just those in the correct sense for the gene.

def plot_igv_view(in_pickle, align_to_site, shift_dict, lower_read_len, upper_read_len, lower_read_len_2, upper_read_len_2):
    file = in_pickle
    in_df = pickle.load(open(file, "rb"))
    all_read_df = in_df["tx_reads"]
    all_properties_df = in_df["tx_properties"]
    print("loaded pickle", file)
    sample_name = in_pickle.split("/")[1].split(".")[0]
    minimum_read_length = 15
    shift = shift_dict[align_to_site]

    transcript = "his3_rep"
    if transcript in gtf_df.transcript_id.values:
        tx_df = gtf_df[gtf_df.transcript_id == transcript]
        tx = tx_df[tx_df.feature == "transcript"]
        exons = tx_df[tx_df.feature == "exon"]
        CDS = tx_df[tx_df.feature == "CDS"]
        strand = tx.strand.item()
        if strand == "+":
            start_codon = CDS.iloc[0].start
            stop_codon = CDS.iloc[-1].end
        elif strand == "-":
            start_codon = CDS.iloc[-1].end
            stop_codon = CDS.iloc[0].start


        hover = HoverTool(tooltips=[
            ("Reads", "@top"),
            ("Position", "@right")
        ], mode="vline", attachment="above", anchor="top_center")

        hover2 = HoverTool(tooltips=[
            ("Position", "@x")
        ], name="triangle")
    
        #28mer
        tx_reads = all_read_df[(all_read_df.transcript_id == transcript) & (lower_read_len <= all_read_df.length) & (all_read_df.length <= upper_read_len)][["left_pos", "len_read_percent"]]
        #shift using riboshift from previous cell
        tx_reads["left_pos"] = tx_reads["left_pos"] + shift
        if len(tx_reads) > 0:
            histo = np.histogram(tx_reads.left_pos, bins=np.arange(tx.start.item(), tx.end.item()+1), )#weights=tx_reads.len_read_percent)
            histo_df = pd.DataFrame({"top":histo[0]/(total_mapped_reads[sample_name]/1e6), "left":histo[1][:-1], "right":histo[1][1:], "bottom":[0]*len(histo[0])})
            histo_df = histo_df[histo_df.top != 0]
            range_upper = histo_df.top.max()*10
        else:
            histo_df = pd.DataFrame()
            range_upper = 1

        read_p = figure(sizing_mode="fixed", y_axis_label="RPM", height=300, width=800, y_range=(0,800), output_backend="svg", tools=["save", hover, "xbox_zoom"])#, y_axis_type="log", y_range = (0.1, range_upper))
        read_p.quad(left="left", right="right", top="top", bottom="bottom", color="#0095c2", source=histo_df)
        read_p.xaxis.ticker = []
        read_p.xgrid.grid_line_color = None
        
        
        #21mer
        tx_reads_2 = all_read_df[(all_read_df.transcript_id == transcript) & (lower_read_len_2 <= all_read_df.length) & (all_read_df.length <= upper_read_len_2)][["left_pos", "len_read_percent"]]
        #shift using riboshift from previous cell
        tx_reads_2["left_pos"] = tx_reads_2["left_pos"] + shift
        if len(tx_reads_2) > 0:
            histo_2 = np.histogram(tx_reads_2.left_pos, bins=np.arange(tx.start.item(), tx.end.item()+1), )#weights=tx_reads.len_read_percent)
            histo_df_2 = pd.DataFrame({"top":histo_2[0]/(total_mapped_reads[sample_name]/1e6), "left":histo_2[1][:-1], "right":histo_2[1][1:], "bottom":[0]*len(histo_2[0])})
            histo_df_2 = histo_df_2[histo_df_2.top != 0]
            range_upper_2 = histo_df_2.top.max()*10
        else:
            histo_df_2 = pd.DataFrame()
            range_upper_2 = 1

        #read_p_2.quad(left="left", right="right", top="top", bottom="bottom", source=histo_df_2, color="#c27b00")
        
        read_p_2 = figure(sizing_mode="fixed", y_axis_label="RPM", height=300, width=800, y_range=(0,800), output_backend="svg", tools=["save", hover, "xbox_zoom"])#, y_axis_type="log", y_range = (0.1, range_upper))
        read_p_2.x_range = read_p.x_range
        read_p_2.quad(left="left", right="right", top="top", bottom="bottom", color="#c27b00", source=histo_df_2)
        read_p_2.xaxis.ticker = []
        read_p_2.xgrid.grid_line_color = None
        
        
        
        
        #gene diagram
        p = figure(sizing_mode="fixed", height=100, width=800, y_range=(-3,3), tools=["xbox_zoom", "reset", hover2, "save"], output_backend="svg")
        p.x_range = read_p.x_range
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        p.yaxis.ticker = []
        p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
        #p.line(x=list(map(int, [tx.start, tx.end+1])), y=[0, 0], color="gray", line_width=5, level="glyph")
        p.quad(left=exons.start, right=exons.end+1, top=1, bottom=-1, color="white", line_color="gray", line_width=2, level="annotation")
        p.quad(left=CDS.start, right=CDS.end+1, top=1, bottom=-1, color="#e8e8e8", line_color="gray", line_width=2, level="annotation")
        p.inverted_triangle(x="x", y="y", color="green", size=10, source=pd.DataFrame({"x":[start_codon], "y":[2]}))
        p.inverted_triangle(x="x", y="y",  color="red", size=10, source=pd.DataFrame({"x":[stop_codon+1], "y":[2]}))
        
        #p.quad(left=start_codon, right=start_codon+1, top=1, bottom=-1, color="green", level="annotation")
        #p.quad(left=stop_codon, right=stop_codon+1, top=1, bottom=-1, color="red", level="annotation")

        read_p.yaxis.major_label_text_font_size = "18pt"
        read_p_2.yaxis.major_label_text_font_size = "18pt"
        read_p.yaxis.ticker.min_interval = 200
        read_p_2.yaxis.ticker.min_interval = 200
        #read_p.min_border_bottom = 30
        #read_p_2.min_border_bottom = 30
        p.xaxis.major_label_text_alpha = 0
        p.xaxis.major_tick_line_alpha = 0
        p.xaxis.minor_tick_line_alpha = 0
        
        read_p.yaxis.axis_label_text_font_size = "20pt"
        read_p_2.yaxis.axis_label_text_font_size = "20pt"
        
        grid = gridplot([[read_p],[read_p_2],[p]], toolbar_location="above")
        return grid#(read_p, read_p_2, p)
    else:
        print(transcript, "not in gtf_df")
    
#for sample in ["wt_100_di_r1", "wt_100_di_r2", "wt_0_di_r1", "wt_0_di_r2"]:
sample = "wt_0_mono_r1"
grid = plot_igv_view(f"./{sample}.pkl", "a_site", shift_dict, 27, 35, 19, 26)
show(grid)
export_svg(grid, filename=f"./recolor_paper_igv_plots/{sample}_igv_plot.png")

loaded pickle ./wt_0_mono_r1.pkl


['./recolor_paper_igv_plots/wt_0_mono_r1_igv_plot.png']

## CGA region 21/28

In [ ]:
load_data_from_pickle("./wt_cga_mono_r2_unmapped_reporteronly.pkl")

loaded pickle ./wt_cga_mono_r2_unmapped_reporteronly.pkl


In [ ]:
all_read_df_1 = all_read_df.copy()

In [ ]:
cga_r1_repeat_reads = all_read_df.query("770 <= left_pos + 17 <= 805").drop_duplicates(subset="read_id")

In [ ]:
cga_r2_repeat_reads = all_read_df.query("770 <= left_pos + 17 <= 805").drop_duplicates(subset="read_id")

In [ ]:
r1_21mer = cga_r1_repeat_reads.groupby("length").count()["read_id"].loc[19:25].sum()
r1_28mer = cga_r1_repeat_reads.groupby("length").count()["read_id"].loc[26:35].sum()
r2_21mer = cga_r2_repeat_reads.groupby("length").count()["read_id"].loc[19:25].sum()
r2_28mer = cga_r2_repeat_reads.groupby("length").count()["read_id"].loc[26:35].sum()

In [ ]:
tot_r1_21mer = all_read_df_1.groupby("length").count()["read_id"].loc[19:25].sum()
tot_r1_28mer = all_read_df_1.groupby("length").count()["read_id"].loc[26:35].sum()
tot_r2_21mer = all_read_df_2.groupby("length").count()["read_id"].loc[19:25].sum()
tot_r2_28mer = all_read_df_2.groupby("length").count()["read_id"].loc[26:35].sum()

In [ ]:
'''r1: 1.0219298245614035
r2: 2.1818181818181817
tot r1: 0.20247497766781125
tot_r2: 0.1880315185336589

To get these numbers use the cga_rx_repeat_reads from wt_cga_mono_rx_unmapped_reporteronly.pkl and do the totals from wt_cga_mono_rx.pkl'''

print("r1:",r1_21mer/r1_28mer)
print("r2:",r2_21mer/r2_28mer)

print("tot r1:",tot_r1_21mer/tot_r1_28mer)
print("tot_r2:",tot_r2_21mer/tot_r2_28mer)

vbar_data = pd.DataFrame([r1_21mer/r1_28mer,r2_21mer/r2_28mer,tot_r1_21mer/tot_r1_28mer,tot_r2_21mer/tot_r2_28mer], index=["CGA_r1", "CGA_r2", "All_r1", "All_r2"])

r1: 1.0219298245614035
r2: 2.1818181818181817
tot r1: 0.20247497766781125
tot_r2: 0.1880315185336589


In [ ]:
vbar_data = vbar_data.T

In [ ]:
vbar_data = vbar_data/(vbar_data.loc[:,["All_r1", "All_r2"]]).mean(axis=1).item()

In [ ]:
vbar_data

,CGA_r1,CGA_r2,All_r1,All_r2
0,5.233868,11.174299,1.036986,0.963014


In [ ]:
#vbar_data = ratios_21_28
#vbar_data.loc[:,['opt_r1', 'non_r1', 'opt_r2', 'non_r2']] = 1/vbar_data[['opt_r1', 'non_r1', 'opt_r2', 'non_r2']]
CGA_mean = vbar_data.loc[0,["CGA_r1", "CGA_r2"]].mean()
All_mean = vbar_data.loc[0,["All_r1", "All_r2"]].mean()
CGA_std = vbar_data.loc[0,["CGA_r1", "CGA_r2"]].std()
All_std = vbar_data.loc[0,["All_r1", "All_r2"]].std()


p = figure(x_range=("CGA region", "All genes"), y_range=(0,13), plot_height=400, plot_width=350, output_backend="svg")
p.vbar(x=["CGA region", "All genes"], top=[CGA_mean, All_mean],
      color="#cfcfcf", width=.75)
p.circle(x=["CGA region"]*2, y=vbar_data.loc[0,["CGA_r1", "CGA_r2"]], color="grey", size=7)
p.circle(x=["All genes"]*2, y=vbar_data.loc[0,["All_r1", "All_r2"]], color="grey", size=7)
p.dash(x=["CGA region"]*2, y=[CGA_mean+CGA_std, CGA_mean-CGA_std], size=15, color="grey")
p.dash(x=["All genes"]*2, y=[All_mean+All_std, All_mean-All_std], size=15, color="grey")
p.line(x=["CGA region"]*2, y=[CGA_mean+CGA_std,CGA_mean-CGA_std], color="grey")
p.line(x=["All genes"]*2, y=[All_mean+All_std, All_mean-All_std], color="grey")

p.title.text = "21mer/28mer ratio"
p.yaxis.axis_label = "Normalized 21mer/28mer ratio"

p.grid.grid_line_alpha = 0
p.yaxis.axis_label_text_font_size = "13pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "13pt"
p.yaxis.minor_tick_line_alpha = 0

show(p)
export_svg(p, filename="./cga_region_21mer_28mer.svg")

['./cga_region_21mer_28mer.svg']

## Disomes

In [ ]:
total_mapped_reads = {
    "wt_cga_di_r1":580096,
    "wt_cga_di_r2": 657546,
    "hel2_cga_di_r1": 844043,
    "hel2_cga_di_r2": 431414,
    "wt_100_di_r1": 660416,
    "wt_100_di_r2": 212522,
    "wt_0_di_r1": 826741,
    "wt_0_di_r2": 1052355,
    "profiling_filtd":1618,
    "wt_100_mono_r1":15340700,
    "wt_0_mono_r1": 7134795,
    "wt_0_mono_r2": 12849527,
    "wt_cga_mono_r1": 11428266,
    "wt_cga_mono_r2": 12508477,
    "hel2_cga_mono_r1": 18489168,
    "hel2_cga_mono_r2": 12819767,
    "wt_cga_di_r3_wt_chx_2_S26_L002_R1_001":3636540,
    "syh1_cga_di_r1":23929,
}

## IGV View
# this has the problem for minus strand reads that it will use the "left side" of the read instead of the 5' end. And it shows reads that 
#   aligned on both strands, rather than just those in the correct sense for the gene.

def plot_igv_disome_view(in_pickle_1, in_pickle_2, align_to_site, shift_dict):
    file = in_pickle_1
    in_df_1 = pickle.load(open("../round2/wt_cga_di_r1.pkl", "rb"))
    all_read_df_1 = in_df_1["tx_reads"]
    all_properties_df = in_df_1["tx_properties"]
    print("loaded pickle", file)
    sample_name_1 = in_pickle_1.split("/")[1].split(".")[0]
    sample_name_2 = in_pickle_2.split("/")[1].split(".")[0]
    minimum_read_length = 15
    shift = shift_dict[align_to_site]

    transcript = "his3_rep"
    if transcript in gtf_df.transcript_id.values:
        tx_df = gtf_df[gtf_df.transcript_id == transcript]
        tx = tx_df[tx_df.feature == "transcript"]
        exons = tx_df[tx_df.feature == "exon"]
        CDS = tx_df[tx_df.feature == "CDS"]
        strand = tx.strand.item()
        if strand == "+":
            start_codon = CDS.iloc[0].start
            stop_codon = CDS.iloc[-1].end
        elif strand == "-":
            start_codon = CDS.iloc[-1].end
            stop_codon = CDS.iloc[0].start


        hover = HoverTool(tooltips=[
            ("Reads", "@top"),
            ("Position", "@right")
        ], mode="vline", attachment="above", anchor="top_center")

        hover2 = HoverTool(tooltips=[
            ("Position", "@x")
        ], name="triangle")
    
    
        #top
        tx_reads = all_read_df_1[(all_read_df_1.transcript_id == transcript)][["left_pos", "len_read_percent"]]
        #shift using riboshift from previous cell
        tx_reads["left_pos"] = tx_reads["left_pos"] + shift
        if len(tx_reads) > 0:
            histo = np.histogram(tx_reads.left_pos, bins=np.arange(tx.start.item(), tx.end.item()+1), )#weights=tx_reads.len_read_percent)
            histo_df = pd.DataFrame({"top":histo[0]/(total_mapped_reads[sample_name_1]/1e6), "left":histo[1][:-1], "right":histo[1][1:], "bottom":[0]*len(histo[0])})
            histo_df = histo_df[histo_df.top != 0]
            range_upper = histo_df.top.max()*10
        else:
            histo_df = pd.DataFrame()
            range_upper = 1

        read_p = figure(y_axis_label="RPM", height=300, width=800, y_range=(0,12000), 
                        output_backend="svg", tools=["save", hover, "xbox_zoom"],
                       x_range = (324, 905))#, y_axis_type="log", y_range = (0.1, range_upper))
        read_p.quad(left="left", right="right", top="top", bottom="bottom", color="#400a69", source=histo_df)
        read_p.xaxis.ticker = []
        read_p.xgrid.grid_line_color = None
        read_p.title.text_font_size = "20pt"
        
        
        #bottom
        file = in_pickle_2
        in_df_2 = pickle.load(open(file, "rb"))
        all_read_df_2 = in_df_2["tx_reads"]
        
        
        tx_reads_2 = all_read_df_2[((all_read_df_2.length > 39) & (all_read_df_2.transcript_id == transcript))][["left_pos", "len_read_percent"]]
        #shift using riboshift from previous cell
        tx_reads_2["left_pos"] = tx_reads_2["left_pos"] + shift
        if len(tx_reads_2) > 0:
            histo_2 = np.histogram(tx_reads_2.left_pos, bins=np.arange(tx.start.item(), tx.end.item()+1), )#weights=tx_reads.len_read_percent)
            histo_df_2 = pd.DataFrame({"top":histo_2[0]/(total_mapped_reads[sample_name_2]/1e6), "left":histo_2[1][:-1], "right":histo_2[1][1:], "bottom":[0]*len(histo_2[0])})
            histo_df_2 = histo_df_2[histo_df_2.top != 0]
            range_upper_2 = histo_df_2.top.max()*10
        else:
            histo_df_2 = pd.DataFrame()
            range_upper_2 = 1

        read_p_2 = figure(y_axis_label="RPM", height=300, width=800, y_range=(0,2000), output_backend="svg", tools=["save", hover, "xbox_zoom"])#, y_axis_type="log", y_range = (0.1, range_upper))
        read_p_2.x_range = read_p.x_range
        read_p_2.quad(left="left", right="right", top="top", bottom="bottom", source=histo_df_2, color="#400a69")
        read_p_2.xaxis.ticker = []
        read_p_2.xgrid.grid_line_color = None
        read_p_2.title.text_font_size = "20pt"
        
        #KDE
        
        read_p.extra_y_ranges = {"kde": Range1d(start=0, end=0.02, max_interval=0.001)}
        read_p.add_layout(LinearAxis(y_range_name="kde", axis_label="KDE"), "right")
        read_p_2.extra_y_ranges = {"kde": Range1d(start=0, end=0.012, max_interval=0.001)}
        read_p_2.add_layout(LinearAxis(y_range_name="kde", axis_label="KDE"), "right")
        
        height = pd.Series(gaussian_kde(tx_reads.left_pos).evaluate(np.arange(start_codon, stop_codon, 1)))#*60*8000
        edge= np.arange(start_codon, stop_codon, 1)
        read_p.line(y=height, x=edge, color="red", line_dash=[10,10], line_width=2, legend_label="KDE", y_range_name="kde")
        
        height = pd.Series(gaussian_kde(tx_reads_2.left_pos).evaluate(np.arange(start_codon, stop_codon, 1)))#*60*14000
        edge= np.arange(start_codon, stop_codon, 1)
        read_p_2.line(y=height, x=edge, color="red", line_dash=[10,10], line_width=2, legend_label="KDE", y_range_name="kde")
        
        print(read_p.x_range.start, read_p.x_range.end)
        
        #gene diagram
        p = figure(height=100, width=800, y_range=(-3,3), tools=["save","xbox_zoom", "reset", hover2], output_backend="svg")
        p.x_range = read_p.x_range
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        p.yaxis.ticker = []
        p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
        #p.line(x=list(map(int, [tx.start, tx.end+1])), y=[0, 0], color="purple", line_width=5, level="glyph")
        p.quad(left=exons.start, right=exons.end+1, top=1, bottom=-1, color="white", line_color="gray", line_width=2, level="annotation")
        p.quad(left=CDS.start, right=CDS.end+1, top=1, bottom=-1, color="#e8e8e8", line_color="gray", line_width=2, level="annotation")
        p.inverted_triangle(x="x", y="y", color="green", size=10, source=pd.DataFrame({"x":[start_codon], "y":[2]}))
        p.inverted_triangle(x="x", y="y",  color="red", size=10, source=pd.DataFrame({"x":[stop_codon+1], "y":[2]}))
        
        #p.quad(left=start_codon, right=start_codon+1, top=1, bottom=-1, color="green", level="annotation")
        #p.quad(left=stop_codon, right=stop_codon+1, top=1, bottom=-1, color="red", level="annotation")
        p.quad(left=770, right=805, color="#990000", top=1, bottom=-1, level="annotation")

        read_p.yaxis.axis_label_text_font_size = "18pt"
        read_p_2.yaxis.axis_label_text_font_size = "18pt"
        read_p.yaxis.major_label_text_font_size = "12pt"
        read_p_2.yaxis.major_label_text_font_size = "12pt"
        read_p.y_range.min_interval = 200
        read_p_2.y_range.min_interval = 200
        read_p.min_border_bottom = 30
        p.xaxis.major_label_text_alpha = 0
        p.xaxis.major_tick_line_alpha = 0
        p.xaxis.minor_tick_line_alpha = 0
        
        read_p.yaxis.axis_label_text_font_size = "20pt"
        read_p_2.yaxis.axis_label_text_font_size = "20pt"
        
        #zoomed in on region before CGA
        #read_p.x_range.start=350
        #read_p.x_range.end=900
        
        
        grid = gridplot([[read_p],[],[read_p_2],[p]])
        return grid
    else:
        print(transcript, "not in gtf_df")
    
#for sample in ["wt_100_di_r1", "wt_100_di_r2", "wt_0_di_r1", "wt_0_di_r2"]:
sample1 = "wt_cga_di_r1"
sample2 = "syh1_cga_di_r1"
grid = plot_igv_disome_view(f"./{sample1}.pkl", f"./{sample2}.pkl", "a_site", {"a_site":17}) #+33
show(grid)
#export_svg(grid, filename="./kde_disome_plot.svg")

loaded pickle ./wt_cga_di_r1.pkl
324 905


## Autocorrelation plot for disomes

In [ ]:
data = pd.read_csv("./disome_autocorrelation.csv")
data = data.apply(lambda col: col/col.max(), axis=0, result_type="broadcast")

n = figure( x_axis_label="Displacement", y_axis_label="Correlation", title="Autocorrelation", tooltips=[("x","@index")],
          plot_width=800, plot_height=500, output_backend="svg")

n.line(y="./hel2_cga_di_r1.pkl", x="index", color="grey", line_width=3, source=data, legend_label="HEL2Δ")
n.line(y="./wt_cga_di_r1.pkl", x="index", color="red", line_width=3, source=data, legend_label="WT")

n.xaxis.ticker.max_interval = 30
n.axis.major_label_text_font_size = "12pt"
n.axis.axis_label_text_font_size = "14pt"
n.title.text_font_size = "14pt"

show(n)
#export_svg(n, filename="./disome_autocorrelation.svg")

### Gene plot with KDE and autocorrelation and fourier transform and signal coherence

In [ ]:
time_series = {}

In [ ]:
## IGV View
# this has the problem for minus strand reads that it will use the "left side" of the read instead of the 5' end. And it shows reads that 
#   aligned on both strands, rather than just those in the correct sense for the gene.



def plot_igv_view(in_pickle, align_to_site, shift_dict):
    
    global time_series
    
    file = in_pickle
    in_df = pickle.load(open(file, "rb"))
    all_read_df = in_df["tx_reads"]
    all_properties_df = in_df["tx_properties"]
    print("loaded pickle", file)

    minimum_read_length = 25

    transcript = "his3_rep"
    if transcript in gtf_df.transcript_id.values:
        tx_df = gtf_df[gtf_df.transcript_id == transcript]
        tx = tx_df[tx_df.feature == "transcript"]
        exons = tx_df[tx_df.feature == "exon"]
        CDS = tx_df[tx_df.feature == "CDS"]
        strand = tx.strand.values
        if strand == "+":
            start_codon = CDS.iloc[0].start
            stop_codon = CDS.iloc[-1].end
        elif strand == "-":
            start_codon = CDS.iloc[-1].end
            stop_codon = CDS.iloc[0].start


        hover = HoverTool(tooltips=[
            ("Reads", "@top"),
            ("Position", "@right")
        ], mode="vline", attachment="above", anchor="top_center")

        hover2 = HoverTool(tooltips=[
            ("Position", "@x")
        ], name="triangle")

        tx_reads = all_read_df[(all_read_df.transcript_id == transcript)][["left_pos", "len_read_percent"]]
        #shift using riboshift from previous cell
        tx_reads["left_pos"] = tx_reads["left_pos"] + shift_dict[align_to_site]
        if len(tx_reads) > 0:
            histo = np.histogram(tx_reads.left_pos, bins=np.arange(tx.start.item(), tx.end.item()+1), weights=tx_reads.len_read_percent)
            histo_df = pd.DataFrame({"top":histo[0]/max(histo[0]), "left":histo[1][:-1], "right":histo[1][1:], "bottom":[0]*len(histo[0])})
            histo_df = histo_df[histo_df.top != 0]
            range_upper = histo_df.top.max()*10
        else:
            histo_df = pd.DataFrame()
            range_upper = 1

        read_p = figure(height=300, width=1500, y_range=(0,1), title=transcript+" - "+file, output_backend="svg", tools=[hover, "xbox_zoom"])#, y_axis_type="log", y_range = (0.1, range_upper))
        read_p.quad(left="left", right="right", top="top", bottom="bottom", color="gray", source=histo_df)
        read_p.xaxis.ticker = []
        read_p.xgrid.grid_line_color = None
        read_p.title.text_font_size = "20pt"
        
        p = figure(height=100, width=1500, y_range=(-3,3), tools=["xbox_zoom", "reset", hover2], output_backend="svg")
        p.x_range = read_p.x_range
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        p.yaxis.ticker = []
        p.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
        p.line(x=list(map(int, [tx.start, tx.end+1])), y=[0, 0], color="purple", line_width=5, level="glyph")
        p.quad(left=exons.start, right=exons.end+1, top=1, bottom=-1, color="white", line_color="purple", line_width=2, level="annotation")
        p.quad(left=CDS.start, right=CDS.end+1, top=1, bottom=-1, color="purple", line_color=None, level="annotation")
        p.inverted_triangle(x="x", y="y", color="green", size=10, source=pd.DataFrame({"x":[start_codon], "y":[2]}))
        p.inverted_triangle(x="x", y="y",  color="red", size=10, source=pd.DataFrame({"x":[stop_codon+1], "y":[2]}))
        
        #esther visualization
        
        ''' pv = figure(height=100, width=1500,y_range=(-3, 3), x_range=(-100,2000), tools=["xbox_zoom", "reset", hover2])
        pv.x_range = read_p.x_range
        pv.quad(top=[1], bottom=[-1], left=[1+1486-1], right=[1896+1486-1], color="white", line_color="purple", line_width=2)
        pv.quad(top=[1], bottom=[-1], left=[201+1486-1], right=[921+1486-1], color="purple", line_color=None, line_width=2)
        pv.quad(top=[1], bottom=[-1], left=[954+1486-1], right=[1011+1486-1], color="blue", line_color=None, line_width=2)
        pv.quad(top=[1], bottom=[-1], left=[1014+1486-1], right=[1038+1486-1], color="green", line_color=None, line_width=2)
        pv.quad(top=[1], bottom=[-1], left=[1311+1486-1], right=[1698+1486-1], color="black", line_color=None, line_width=2)

        pv.inverted_triangle(x="x", y="y", color="green", size=10, source=pd.DataFrame({"x":[1008+1486-1], "y":[2]}))
        
        pv.xgrid.grid_line_color = None
        pv.ygrid.grid_line_color = None
        pv.yaxis.ticker = []
        pv.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
        
        text = ["GFP","P2A","FL","NONOPT HIS3"]
        source = ColumnDataSource(dict(x=[500+1486-1,960+1486-1,1015+1486-1,1400+1486-1], y=[-0.7,-0.7,-0.7,-0.7], text=text))
        glyph = Text(x="x", y="y", text="text", text_color="white")
        pv.add_glyph(source, glyph)'''
        
        p.quad(left=770, right=805, color="blue", top=1, bottom=-1, level="annotation")


        p.quad(left=stop_codon, right=stop_codon+1, top=1, bottom=-1, color="red", level="annotation")
        
        #KDE
        height = pd.Series(gaussian_kde(tx_reads.left_pos).evaluate(np.arange(start_codon, stop_codon, 1)))*60
        edge= np.arange(start_codon, stop_codon, 1)
        read_p.line(y=height, x=edge, color="red", line_dash=[10,10], line_width=3)
        
        #autocorrelation
        n = figure(x_range=(0,100), x_axis_label="Displacement", y_axis_label="Corelation", title="Autocorrelation", tooltips=[("x","@x"), ("y","@y")], width=450, height=450,
                  output_backend="svg", y_range=(0,9))
        
        def autocorr(x): # from https://ipython-books.github.io/103-computing-the-autocorrelation-of-a-time-series/
            result = np.correlate(x, x, mode='full')
            return result[result.size // 2:]
        
        y = autocorr(histo_df.top)
        n.line(y=y, x=range(0,len(y)), color="#ad494a", line_width=3) # syh1 #9467bd, hel2 #ad494a, wt #2ca02c
        
        n.xaxis.major_label_text_font_size = "40pt"
        n.yaxis.major_label_text_font_size = "50pt"
        n.axis.axis_label_text_font_size = "20pt"
        #n.yaxis.ticker.desired_num_ticks = 2
        n.yaxis.ticker.min_interval = 1
        n.title.text = ""
        
        show(n)
        
        
        #FFT
        '''histo_df = histo_df[(histo_df["left"] >= 474) & (histo_df["right"] <= 787)]
        ft = pd.DataFrame()

        ft["amplitudes"] = (np.abs(np.fft.fft(histo_df["top"])))**2
        ft["angles"] = np.angle(np.fft.fft(histo_df["top"]))
        ft["fftfreq"] = 1/np.fft.fftfreq(histo_df["top"].size)

        hist2, edges2 = np.histogram(ft["amplitudes"], bins=999)
        source1dict = {"left":edges2[:-1], "right":edges2[1:], "top":hist2}

        hover1 = HoverTool(tooltips=[
            ("freq", "@fftfreq"),
            ("amplitude", "@amplitudes"),
            
            ])

        source1 = ColumnDataSource(pd.DataFrame.from_dict(source1dict))

        p1 = figure(tools=[hover1,"pan","box_zoom","reset", "save", "wheel_zoom"], title="Fourier Transform",
                    background_fill_color="white", x_range=(0,100), #y_range=(0,50000000),
                    x_axis_label="Period", y_axis_label="Power (Amplitude^2)")
        p1.line(y="amplitudes", x="fftfreq", source=ft, line_width=2)
        p1.yaxis.axis_label_text_font_size="15pt"
        p1.xaxis.axis_label_text_font_size="15pt"
        p1.title.text_font_size="15pt"
        p1.yaxis.major_label_text_font_size="12pt"
        p1.xaxis.major_label_text_font_size="12pt"

        show(p1)
        
        time_series[in_pickle] = ft'''

        grid = gridplot([[read_p]])#,[p]])
        show(grid)
        return (n, grid)#grid
    else:
        print(transcript, "not in gtf_df")


sample = "hel2_cga_mono_r2"        
grid = plot_igv_view(f"../round2/{sample}.pkl", "a_site", shift_dict)
#show(grid)
export_svgs(grid[0], filename=f"./paper_plots/{sample}_autocorr.svg")
export_svgs(grid[1], filename=f"./paper_plots/{sample}_geneplot.svg")

loaded pickle ../round2/hel2_cga_mono_r2.pkl


['./paper_plots/hel2_cga_mono_r2_geneplot.svg']